In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import os

In [ ]:
gdp_df = pd.read_csv(r"/content/drive/MyDrive/CS_Finance_Data/MacroEconomic_Variables/GDP.csv", parse_dates=True)
gdp_df['DATE'] = pd.to_datetime(gdp_df['DATE'])
print(gdp_df.shape)
gdp_df.head()

(81, 2)


,DATE,GDP
0,2004-01-01,11923.447
1,2004-04-01,12112.815
2,2004-07-01,12305.307
3,2004-10-01,12527.214
4,2005-01-01,12767.286


In [ ]:
uner_df = pd.read_csv(r"/content/drive/MyDrive/CS_Finance_Data/MacroEconomic_Variables/UNRATE.csv")
uner_df['DATE'] = pd.to_datetime(uner_df['DATE'])
print(uner_df.shape)
uner_df.head()

(82, 2)


,DATE,UNRATE
0,2004-01-01,5.700000
1,2004-04-01,5.600000
2,2004-07-01,5.433333
3,2004-10-01,5.433333
4,2005-01-01,5.300000


In [ ]:
df_gdp_uner = pd.merge(gdp_df,uner_df,on='DATE')
print(df_gdp_uner.shape)
df_gdp_uner.head()

(81, 3)


,DATE,GDP,UNRATE
0,2004-01-01,11923.447,5.700000
1,2004-04-01,12112.815,5.600000
2,2004-07-01,12305.307,5.433333
3,2004-10-01,12527.214,5.433333
4,2005-01-01,12767.286,5.300000


In [ ]:
df_gdp_uner.rename(columns={'DATE': 'date'}, inplace=True)

In [ ]:
def update_quart_enddate_shift(df):
    # Define a dictionary to map the month to the new date
    quarter_end_dates = {1: (12, 31), 4: (3, 31), 7: (6, 30), 10: (9, 30)}

    # Apply the mapping to update the dates
    for month, (new_month, new_day) in quarter_end_dates.items():
        df.loc[df['date'].dt.month == month, 'date'] = df['date'].apply(
            lambda x: x.replace(month=new_month, day=new_day) if x.month == month else x
        )

    # offset the years by 1 for those whose month is december
    tmp_ids = df[df["date"].dt.month == 12].index
    df.loc[tmp_ids, "date"] = df.loc[tmp_ids, "date"] - pd.DateOffset(years=1)

    # drop the first row df_gdp_uner
    df.drop(df.index[0], inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [ ]:
df_gdp_uner =  update_quart_enddate_shift(df_gdp_uner)
df_gdp_uner.head()

,date,GDP,UNRATE
0,2004-03-31,12112.815,5.600000
1,2004-06-30,12305.307,5.433333
2,2004-09-30,12527.214,5.433333
3,2004-12-31,12767.286,5.300000
4,2005-03-31,12922.656,5.100000


In [ ]:
# # add a year column to df_gdp_uner
# df_gdp_uner['Year'] = df_gdp_uner['date'].dt.year
# df_gdp_uner.head()

In [ ]:
# Assign quarter
# def assign_quarter(date):
#     month = date.month
#     if month == 3:
#         return 'Q1'
#     elif month == 6:
#         return 'Q2'
#     elif month == 9:
#         return 'Q3'
#     else:
#         return 'Q4'


In [ ]:
# df_gdp_uner['Quarter'] = df_gdp_uner['date'].apply(assign_quarter)
# df_gdp_uner.head()

In [ ]:
rates_df = pd.read_csv(r"/content/drive/MyDrive/CS_Finance_Data/MacroEconomic_Variables/yield-curve-rates-2004-2024.csv", encoding='latin1')
rates_df.head()

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,07/15/2024,5.48,5.51,5.43,5.40,5.23,4.85,4.44,4.23,4.13,4.16,4.23,4.56,4.46
1,07/12/2024,5.47,5.52,5.43,5.41,5.23,4.87,4.45,4.22,4.10,4.13,4.18,4.50,4.39
2,07/11/2024,5.48,5.53,5.44,5.41,5.25,4.91,4.50,4.26,4.13,4.15,4.20,4.51,4.41
3,07/10/2024,5.46,5.50,5.46,5.46,5.33,5.01,4.62,4.38,4.24,4.24,4.28,4.58,4.47
4,07/09/2024,5.45,5.51,5.46,5.46,5.34,5.02,4.62,4.37,4.24,4.25,4.30,4.59,4.49


In [ ]:
# Changing Date to Date-time Format
rates_df['Date'] = pd.to_datetime(rates_df['Date'].astype(str), format='mixed', utc=True)
rates_df.head()

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2024-07-15 00:00:00+00:00,5.48,5.51,5.43,5.40,5.23,4.85,4.44,4.23,4.13,4.16,4.23,4.56,4.46
1,2024-07-12 00:00:00+00:00,5.47,5.52,5.43,5.41,5.23,4.87,4.45,4.22,4.10,4.13,4.18,4.50,4.39
2,2024-07-11 00:00:00+00:00,5.48,5.53,5.44,5.41,5.25,4.91,4.50,4.26,4.13,4.15,4.20,4.51,4.41
3,2024-07-10 00:00:00+00:00,5.46,5.50,5.46,5.46,5.33,5.01,4.62,4.38,4.24,4.24,4.28,4.58,4.47
4,2024-07-09 00:00:00+00:00,5.45,5.51,5.46,5.46,5.34,5.02,4.62,4.37,4.24,4.25,4.30,4.59,4.49


In [ ]:
# Giving the Year and Quarter each entry
rates_df['Year'] = rates_df['Date'].dt.year
rates_df['Quarter'] = rates_df['Date'].dt.quarter
rates_df.sort_values('Date').head()

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,Year,Quarter
5138,2004-01-02 00:00:00+00:00,0.88,NaN,0.93,NaN,1.02,1.31,1.94,2.47,3.36,3.90,4.38,5.21,NaN,2004,1
5137,2004-01-05 00:00:00+00:00,0.88,NaN,0.91,NaN,1.05,1.35,1.95,2.51,3.39,3.92,4.41,5.23,NaN,2004,1
5136,2004-01-06 00:00:00+00:00,0.88,NaN,0.91,NaN,1.03,1.30,1.84,2.38,3.26,3.80,4.29,5.13,NaN,2004,1
5135,2004-01-07 00:00:00+00:00,0.88,NaN,0.91,NaN,1.02,1.29,1.84,2.36,3.25,3.76,4.27,5.11,NaN,2004,1
5134,2004-01-08 00:00:00+00:00,0.87,NaN,0.88,NaN,1.01,1.29,1.85,2.37,3.24,3.76,4.27,5.12,NaN,2004,1


In [ ]:
finalrates = rates_df.groupby(['Year', 'Quarter']).mean()
finalrates = finalrates.drop(columns=["Date"])
finalrates.head(20)

1 Mo  2 Mo      3 Mo  4 Mo      6 Mo      1 Yr      2 Yr  \
Year Quarter                                                                 
2004 1        0.909677   NaN  0.933710   NaN  1.003548  1.221452  1.686613   
     2        0.954516   NaN  1.096129   NaN  1.359677  1.775484  2.454677   
     3        1.370000   NaN  1.513906   NaN  1.788437  2.075000  2.556563   
     4        1.841774   NaN  2.047419   NaN  2.297581  2.472742  2.822419   
2005 1        2.360984   NaN  2.589180   NaN  2.876885  3.072459  3.454918   
     2        2.709687   NaN  2.926719   NaN  3.179688  3.337344  3.645938   
     3        3.228594   NaN  3.438281   NaN  3.707031  3.791094  3.957969   
     4        3.702131   NaN  3.913607   NaN  4.250492  4.288525  4.364590   
2006 1        4.358710   NaN  4.506613   NaN  4.657258  4.640323  4.604677   
     2        4.674921   NaN  4.830159   NaN  5.033651  5.021429  4.997937   
     3        4.953492   NaN  5.035079   NaN  5.172540  5.091111  4.929048   
     4        5.020161   NaN  5.031774   NaN  5.113387  4.988226  4.737581   
2007 1        5.107742   NaN  5.114032   NaN  5.133065  5.007581  4.760484   
     2        4.776719   NaN  4.872031   NaN  5.000000  4.934219  4.803750   
     3        4.278889   NaN  4.434286   NaN  4.607778  4.535079  4.390794   
     4        3.462097   NaN  3.491613   NaN  3.709194  3.635000  3.489677   
2008 1        2.130820   NaN  2.104426   NaN  2.158361  2.112951  2.029344   
     2        1.511094   NaN  1.650156   NaN  1.872500  2.065937  2.416406   
     3        1.392656   NaN  1.520312   NaN  1.863125  2.126406  2.360000   
     4        0.141290   NaN  0.327627   NaN  0.740968  0.989194  1.215484   

                  3 Yr      5 Yr      7 Yr     10 Yr     20 Yr     30 Yr  
Year Quarter                                                              
2004 1        2.163065  2.980968  3.501452  4.010000  4.877903       NaN  
     2        2.971935  3.720968  4.183871  4.597097  5.355645       NaN  
     3        2.919531  3.505625  3.922188  4.301406  5.068281       NaN  
     4        3.058871  3.493710  3.855000  4.175806  4.874839       NaN  
2005 1        3.623934  3.890820  4.098852  4.303607  4.764098       NaN  
     2        3.732344  3.872969  3.984688  4.158594  4.548750       NaN  
     3        3.989219  4.039375  4.108437  4.215156  4.507656       NaN  
     4        4.372295  4.391148  4.425738  4.488197  4.767541       NaN  
2006 1        4.582581  4.552419  4.555484  4.577097  4.774032  4.663056  
     2        4.987302  4.993175  5.018095  5.072698  5.290635  5.143016  
     3        4.868413  4.842222  4.850000  4.893492  5.085397  4.994286  
     4        4.648710  4.601774  4.603548  4.630645  4.832903  4.741935  
2007 1        4.679677  4.643387  4.649677  4.679032  4.890806  4.796452  
     2        4.763594  4.760781  4.783750  4.846094  5.069063  4.988906  
     3        4.416508  4.512698  4.602857  4.738413  5.012540  4.948889  
     4        3.513387  3.797581  3.991613  4.266613  4.659032  4.612258  
2008 1        2.170984  2.751475  3.152131  3.665246  4.399016  4.412131  
     2        2.659063  3.155000  3.458281  3.881562  4.587031  4.574063  
     3        2.632969  3.112812  3.437031  3.862188  4.495625  4.449219  
     4        1.480968  2.172742  2.620806  3.234677  3.946290  3.661774

In [ ]:
finalrates.reset_index(inplace=True)
finalrates.head()

,Year,Quarter,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2004,1,0.909677,NaN,0.933710,NaN,1.003548,1.221452,1.686613,2.163065,2.980968,3.501452,4.010000,4.877903,NaN
1,2004,2,0.954516,NaN,1.096129,NaN,1.359677,1.775484,2.454677,2.971935,3.720968,4.183871,4.597097,5.355645,NaN
2,2004,3,1.370000,NaN,1.513906,NaN,1.788437,2.075000,2.556563,2.919531,3.505625,3.922188,4.301406,5.068281,NaN
3,2004,4,1.841774,NaN,2.047419,NaN,2.297581,2.472742,2.822419,3.058871,3.493710,3.855000,4.175806,4.874839,NaN
4,2005,1,2.360984,NaN,2.589180,NaN,2.876885,3.072459,3.454918,3.623934,3.890820,4.098852,4.303607,4.764098,NaN


In [ ]:
cmev = pd.concat([finalrates, df_gdp_uner], axis = 1)
cmev.head()

,Year,Quarter,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,date,GDP,UNRATE
0,2004,1,0.909677,NaN,0.933710,NaN,1.003548,1.221452,1.686613,2.163065,2.980968,3.501452,4.010000,4.877903,NaN,2004-03-31,12112.815,5.600000
1,2004,2,0.954516,NaN,1.096129,NaN,1.359677,1.775484,2.454677,2.971935,3.720968,4.183871,4.597097,5.355645,NaN,2004-06-30,12305.307,5.433333
2,2004,3,1.370000,NaN,1.513906,NaN,1.788437,2.075000,2.556563,2.919531,3.505625,3.922188,4.301406,5.068281,NaN,2004-09-30,12527.214,5.433333
3,2004,4,1.841774,NaN,2.047419,NaN,2.297581,2.472742,2.822419,3.058871,3.493710,3.855000,4.175806,4.874839,NaN,2004-12-31,12767.286,5.300000
4,2005,1,2.360984,NaN,2.589180,NaN,2.876885,3.072459,3.454918,3.623934,3.890820,4.098852,4.303607,4.764098,NaN,2005-03-31,12922.656,5.100000
